In [127]:
import numpy as np
import pandas as pd
from statsmodels import regression
import statsmodels.api as sm

In [128]:
eth = pd.read_csv("data 041018/eth.csv")

In [129]:
list(eth.columns.values)

['date',
 'txVolume(USD)',
 'adjustedTxVolume(USD)',
 'txCount',
 'marketcap(USD)',
 'price(USD)',
 'exchangeVolume(USD)',
 'generatedCoins',
 'fees',
 'activeAddresses',
 'medianTxValue(USD)',
 'medianFee',
 'averageDifficulty',
 'paymentCount',
 'blockSize',
 'blockCount']

In [130]:
##remove all nan
eth = eth[np.isfinite(eth['txVolume(USD)'])]

In [131]:
eth = eth.set_index(eth['date']).drop('date',axis='columns')

In [132]:
eth['future_ma5']=np.nan

In [135]:
for i in range(len(eth['future_ma5'])-5):
    eth['future_ma5'][i]=np.average(eth['price(USD)'][i+1:i+6])
    

/Users/See/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [136]:
eth = eth.drop(eth.index[-5:],axis='index')

In [137]:
response = eth['future_ma5']
data = eth.drop('future_ma5',axis='columns')

print((response.shape), (data.shape))

(1148,) (1148, 15)


In [138]:
def forward_aic(response, data):
    #define some variables
    explanatory = list(data.columns)
    selected = pd.Series(np.ones(data.shape[0]), index=data.index, name="Intercept")
    current_score, best_new_score = np.inf, np.inf
    
    # Loop to find the best model
    while current_score == best_new_score and len(explanatory) != 0:
        
        scores_with_elements = []
        count = 0
        
        # For each explanatory variable
        for element in explanatory:
            # Make a set of explanatory variables including our current best and the new one
            tmp = pd.concat([selected, data[element]], axis=1)
            # Test the set
            result = regression.linear_model.OLS(list(response), tmp).fit()
            score = result.aic
            scores_with_elements.append((score, element, count))
            count += 1
        
        # Sort the scoring list
        scores_with_elements.sort(reverse = True)
        # Get the best new variable
        best_new_score, best_element, index = scores_with_elements.pop()
        if current_score > best_new_score:
            # If it's better than the best add it to the set
            explanatory.pop(index)
            selected = pd.concat([selected, data[best_element]],axis=1)
            current_score = best_new_score
    # Return the final model
    model = regression.linear_model.OLS(list(response), selected).fit()
    return model

In [139]:
result = forward_aic(response, data)
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                     8571.
Date:                Tue, 09 Oct 2018   Prob (F-statistic):               0.00
Time:                        11:40:28   Log-Likelihood:                -5523.0
No. Observations:                1148   AIC:                         1.107e+04
Df Residuals:                    1135   BIC:                         1.114e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept               -11.3370      2.373     -4.777      0.000     -15.994      -6.680
price(USD)                2.9088      0.271     10.735      0.000       2.377       3.440
paymentCount              0.0004   4.53e-05      8.207      0.000       0.000       0.000
marketcap(USD)        -2.098e-08    2.8e-09     -7.500      0.000   -2.65e-08   -1.55e-08
txVolume(USD)         -5.148e-09   7.18e-10     -7.174      0.000   -6.56e-09   -3.74e-09
medianTxValue(USD)       -0.1070      0.039     -2.735      0.006      -0.184      -0.030
exchangeVolume(USD)    2.133e-08   2.64e-09      8.067      0.000    1.61e-08    2.65e-08
adjustedTxVolume(USD) -2.661e-08   4.65e-09     -5.727      0.000   -3.57e-08   -1.75e-08
activeAddresses          -0.0005   7.12e-05     -6.932      0.000      -0.001      -0.000
txCount                   0.0002   3.88e-05      6.335      0.000       0.000       0.000
medianFee              5381.7731   2396.964      2.245      0.025     678.794    1.01e+04
blockSize             -3.276e-07   9.63e-08     -3.401      0.001   -5.17e-07   -1.39e-07
fees                      0.0070      0.003      2.437      0.015       0.001       0.013
==============================================================================
Omnibus:                      289.175   Durbin-Watson:                   0.549
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6449.570
Skew:                          -0.599   Prob(JB):                         0.00
Kurtosis:                      14.550   Cond. No.                     9.51e+13
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.51e+13. This might indicate that there are
strong multicollinearity or other numerical problems.
"""